In [1]:
from google.cloud import bigquery
import gc
import pandas as pd
import numpy as np
import datetime
import os
import json
import glob
from haversine import haversine
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/mnt/clients/juba/hqjubaapp02/sharefolder/GoodYear/docs/merkle-gdyr-prod-9783607c77a1.json"

zip_centers=json.load(open("/mnt/clients/juba/hqjubaapp02/sharefolder/Docs/Geo_mapping/updated_zip_centers_JL_2019-05-23.json"))
client = bigquery.Client()

print(datetime.datetime.now())
print(os.getcwd())

2020-11-06 13:40:12.115871
/mnt/clients/juba/hqjubaapp02/sharefolder/GoodYear/jian/Audience_selections


In [2]:
dict_date_range={"R1H":['2019-10-01','2020-09-30'],
                 "R2M":['2017-04-01','2019-09-30'],
                 "R3L":[None,'2017-03-01']}

In [3]:
query_ecom="""
            select indiv_key, max(invoice_date) as last_trans_date from mdb.trn_detail
            where store_id='ECOM' and indiv_key is not null
            group by indiv_key
            """

df_ids_ECOM_all=client.query(query_ecom).result().to_dataframe()
print(df_ids_ECOM_all.shape)
df_ids_ECOM_all=df_ids_ECOM_all.sort_values(['indiv_key','last_trans_date'],ascending=[True,False]).drop_duplicates('indiv_key')
print(df_ids_ECOM_all.shape)

/home/jliang/.local/lib/python3.7/site-packages/google/cloud/bigquery/client.py:441: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


(308097, 2)
(308097, 2)


In [4]:
query_crm="""
select t1.indiv_key, t1.best_email_txt, t1.best_phone_num, t2.postal_cd, t2.state_province_cd, t2.country_cd from mdb.indiv_summary as t1
join mdb.addr as t2 on
t1.cr_addr_key=t2.cr_addr_key
"""

print(query_crm)

df_all_ids=client.query(query_crm).result().to_dataframe()
print(df_all_ids.shape,df_all_ids['indiv_key'].nunique())


select t1.indiv_key, t1.best_email_txt, t1.best_phone_num, t2.postal_cd, t2.state_province_cd, t2.country_cd from mdb.indiv_summary as t1
join mdb.addr as t2 on
t1.cr_addr_key=t2.cr_addr_key

(7538892, 6) 7538892


In [5]:
df_all_ids['best_email_txt']=df_all_ids['best_email_txt'].apply(lambda x: x.lower().strip() if pd.notnull(x) else x)
df_all_ids['best_phone_num']=df_all_ids['best_phone_num'].apply(lambda x: "1"+str(int(x)).lower().strip() if pd.notnull(x) else x)
df_all_ids.head(2)

,indiv_key,best_email_txt,best_phone_num,postal_cd,state_province_cd,country_cd
0,1222782744000131,lelsinger_53065@yahoo.com,None,54935,WI,US
1,1222781549002871,michenerheather@yahoo.com,17145536855,92648,CA,US


In [6]:
df_all_ids['best_phone_num'].apply(lambda x: len(x) if pd.notnull(x) else x).unique()

array([nan, 11.])

In [7]:
df_HI_AK_ids=df_all_ids[df_all_ids['state_province_cd'].isin(["HI","AK"])]
list_HI_AK_ids=df_HI_AK_ids[pd.notnull(df_HI_AK_ids['indiv_key'])]['indiv_key'].unique().tolist()
list_HI_AK_emails=df_HI_AK_ids[pd.notnull(df_HI_AK_ids['best_email_txt'])]['best_email_txt'].unique().tolist()
list_HI_AK_phones=df_HI_AK_ids[pd.notnull(df_HI_AK_ids['best_phone_num'])]['best_phone_num'].unique().tolist()

print(df_HI_AK_ids.shape)

(242545, 6)


In [8]:
df_HI_AK_ids.head(2)

,indiv_key,best_email_txt,best_phone_num,postal_cd,state_province_cd,country_cd
18,1222785907000692,cmzj122614@gmail.com,None,96813,HI,US
85,1222782721003319,tubbzzz@hotmail.com,18088965530,96720,HI,US


In [9]:
final_uploaded_folder="/mnt/clients/juba/hqjubaapp02/sharefolder/GoodYear/jian/Audience_selections/output_cleaned_audience_2020-11-06/"
list_uploaded_main_aud_files=glob.glob(final_uploaded_folder+"*.csv")

list_already_defined_ids=[]
list_already_defined_emails=[]
list_already_defined_phones=[]
i=0

for file in list_uploaded_main_aud_files:
    df=pd.read_csv(file,dtype=str)
    list_id_file=df[pd.notnull(df['indiv_key'])]['indiv_key'].unique().tolist()
    list_email_file=df[pd.notnull(df['best_email_txt'])]['best_email_txt'].unique().tolist()
    list_phone_file=df[pd.notnull(df['best_phone_num'])]['best_phone_num'].unique().tolist()
    
    list_already_defined_ids=list_already_defined_ids+list_id_file
    list_already_defined_emails=list_already_defined_emails+list_email_file
    list_already_defined_phones=list_already_defined_phones+list_phone_file
    
    i+=1
    if i%10==1:
        list_already_defined_ids=list(set(list_already_defined_ids))
        list_already_defined_emails=list(set(list_already_defined_emails))
        list_already_defined_phones=list(set(list_already_defined_phones))



In [10]:
list_already_defined_ids=list(set(list_already_defined_ids))
list_already_defined_emails=list(set(list_already_defined_emails))
list_already_defined_phones=list(set(list_already_defined_phones))

print(len(list_already_defined_ids))
print(len(list_already_defined_emails))
print(len(list_already_defined_phones))

816190
328351
785626


In [11]:
df_ids_ECOM_all=pd.merge(df_ids_ECOM_all,df_all_ids,on="indiv_key",how="left")
print(df_ids_ECOM_all.shape)
df_ids_ECOM_all=df_ids_ECOM_all[~df_ids_ECOM_all['indiv_key'].isin(list_already_defined_ids)]
df_ids_ECOM_all=df_ids_ECOM_all[~df_ids_ECOM_all['best_email_txt'].isin(list_already_defined_emails)]
df_ids_ECOM_all=df_ids_ECOM_all[~df_ids_ECOM_all['best_phone_num'].isin(list_already_defined_phones)]
print(df_ids_ECOM_all.shape)
df_ids_ECOM_all=df_ids_ECOM_all[~df_ids_ECOM_all['indiv_key'].isin(list_HI_AK_ids)]
df_ids_ECOM_all=df_ids_ECOM_all[~df_ids_ECOM_all['best_email_txt'].isin(list_HI_AK_emails)]
df_ids_ECOM_all=df_ids_ECOM_all[~df_ids_ECOM_all['best_phone_num'].isin(list_HI_AK_phones)]
print(df_ids_ECOM_all.shape)


(308097, 7)
(252126, 7)
(249091, 7)


In [12]:
dict_date_range={"R1H":['2019-10-01','2020-09-30'],
                 "R2M":['2017-04-01','2019-09-30'],
                 "R3L":[None,'2017-03-31']}

df_ids_ECOM_all=df_ids_ECOM_all.sort_values(['indiv_key','last_trans_date'],ascending=[True,False]).drop_duplicates('indiv_key')
df_ids_ECOM_all['last_trans_date']=df_ids_ECOM_all['last_trans_date'].astype(str).str[:10]
df_ids_ECOM_all=df_ids_ECOM_all[df_ids_ECOM_all['last_trans_date']<=dict_date_range['R1H'][1]]

df_ids_ECOM_all['recency_group']=np.where(df_ids_ECOM_all['last_trans_date']<=dict_date_range['R3L'][1],"R3L",
                                         np.where(df_ids_ECOM_all['last_trans_date']<=dict_date_range['R2M'][1],"R2M","R1H")
                                         )

agg_func={"last_trans_date":["min","max","count"]}
df_qc=df_ids_ECOM_all.groupby("recency_group").agg(agg_func).reset_index()
df_qc

recency_group last_trans_date                    
                            min         max   count
0           R1H      2019-10-01  2020-09-30   83911
1           R2M      2017-04-01  2019-09-30  128116
2           R3L      2015-04-15  2017-03-31   25304

In [13]:
df_ids_ECOM_all['segment_name']=df_ids_ECOM_all['recency_group'].apply(lambda x: "segment_TorC_TireAndService_Balance_Ecommerce_All_"+x)
df_ids_ECOM_all.head(3)

,indiv_key,last_trans_date,best_email_txt,best_phone_num,postal_cd,state_province_cd,country_cd,recency_group,segment_name
0,1222780000000004,2018-05-14,chrismaynard104@gmail.com,19049454124,32221,FL,US,R2M,segment_TorC_TireAndService_Balance_Ecommerce_...
1,1222780000000012,2016-08-01,mredwardarroyo@gmail.com,17145526630,92707,CA,US,R3L,segment_TorC_TireAndService_Balance_Ecommerce_...
3,1222780000000036,2015-11-06,a1965m@aol.com,19738843336,07054,NJ,US,R3L,segment_TorC_TireAndService_Balance_Ecommerce_...


In [14]:
import random
random.seed(0)
control_pctg=0.2
df_summary_C=pd.DataFrame()
df_summary_T=pd.DataFrame()

for segment_name, df_group in df_ids_ECOM_all.groupby("segment_name"):
    df_len=df_group.shape[0]
    print(segment_name,df_len)
    list_C_index=random.sample(range(df_len),int(np.ceil(df_len*control_pctg)))
    df_group=df_group.reset_index()
    del df_group['index']
    df_group=df_group.reset_index()
    
    df_group_C=df_group[df_group['index'].isin(list_C_index)]
    df_group_T=df_group[~df_group['index'].isin(list_C_index)]
    
    segment_name_C=segment_name.replace("_TorC_","_Control_")
    segment_name_T=segment_name.replace("_TorC_","_Test_")
    
    df_group_C['segment_name']=segment_name_C
    df_group_T['segment_name']=segment_name_T
    
    print(segment_name_C,df_group_C.shape,df_group_C['indiv_key'].nunique(),df_group_C['best_email_txt'].nunique(),df_group_C['best_phone_num'].nunique())
    print(segment_name_T,df_group_T.shape,df_group_T['indiv_key'].nunique(),df_group_T['best_email_txt'].nunique(),df_group_T['best_phone_num'].nunique())
    print("\n")
    
    df_1_row_C=pd.DataFrame({"segment":segment_name_C,
                             "n_unique_key":df_group_C['indiv_key'].nunique(),
                             "n_unique_email":df_group_C['best_email_txt'].nunique(),
                             "n_unique_phone":df_group_C['best_phone_num'].nunique()
                            },index=[0])
    df_1_row_T=pd.DataFrame({"segment":segment_name_T,
                         "n_unique_key":df_group_T['indiv_key'].nunique(),
                         "n_unique_email":df_group_T['best_email_txt'].nunique(),
                         "n_unique_phone":df_group_T['best_phone_num'].nunique()
                        },index=[0])
    
    df_summary_C=df_summary_C.append(df_1_row_C)
    df_summary_T=df_summary_T.append(df_1_row_T)
    
    df_group_C.to_csv(final_uploaded_folder+segment_name_C+".csv",index=False)
    df_group_T.to_csv(final_uploaded_folder+segment_name_T+".csv",index=False)
    
    

segment_TorC_TireAndService_Balance_Ecommerce_All_R1H 83911
segment_Control_TireAndService_Balance_Ecommerce_All_R1H (16783, 10) 16783 16378 16330
segment_Test_TireAndService_Balance_Ecommerce_All_R1H (67128, 10) 67128 64954 64989




/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


segment_TorC_TireAndService_Balance_Ecommerce_All_R2M 128116
segment_Control_TireAndService_Balance_Ecommerce_All_R2M (25624, 10) 25624 24344 25058
segment_Test_TireAndService_Balance_Ecommerce_All_R2M (102492, 10) 102492 96463 99601


segment_TorC_TireAndService_Balance_Ecommerce_All_R3L 25304
segment_Control_TireAndService_Balance_Ecommerce_All_R3L (5061, 10) 5061 4659 5051
segment_Test_TireAndService_Balance_Ecommerce_All_R3L (20243, 10) 20243 18538 20145




In [15]:
query_ecom="""
            select count(distinct indiv_key) from mdb.trn_detail
            """

df=client.query(query_ecom).result().to_dataframe()
df

/home/jliang/.local/lib/python3.7/site-packages/google/cloud/bigquery/client.py:441: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


,f0_
0,7414313


In [16]:
df_summary_T.to_csv("./df_summary_Test_TireAndService_Balance_Ecommerce_All.csv",index=False)
df_summary_C.to_csv("./df_summary_Control_TireAndService_Balance_Ecommerce_All.csv",index=False)